# Evaluation

In [5]:
import pandas as pd
import requests

SCHEMALESS_URL = "http://localhost:5001/solr/articles/select"
SCHEMA_URL = "http://localhost:5002/solr/articles/select"


In [27]:
# METRICS TABLE
# Define custom decorator to automatically calculate metric based on key
metrics = {}
def metric(f): return metrics.setdefault(f.__name__, f)


@metric
def ap(results, relevant):
    """Average Precision"""
    precision_values = [
        len([
            doc
            for doc in results[:idx]
            if doc in relevant
        ]) / idx
        for idx in range(1, len(results))
    ]
    if len(precision_values) == 0:
        return 0
    return sum(precision_values)/len(precision_values)


@metric
def p10(results, relevant, n=10):
    """Precision at N"""
    return len([doc for doc in results[:n] if doc in relevant])/n

@metric
def re(results, relevant):
    """Recall"""
    return len([doc for doc in results if doc in relevant])/len(relevant)


def calculate_metric(key, results, relevant):
    return metrics[key](results, relevant)


# Define metrics to be calculated
evaluation_metrics = {
    'ap': 'Average Precision',
    'p10': 'Precision at 10 (P@10)',
    're': 'Recall',
}


def _parse_articles(articles):
    return list(map(lambda x: "{}/{}/{}".format(x["book"], x["key"], x["date"]), articles))

def metrics_table(title, search_results, qrels_results):
    search_results = _parse_articles(search_results)

    print(title)
    # Calculate all metrics and export results as LaTeX table
    df = pd.DataFrame([['Metric', 'Value']] +
                      [
        [evaluation_metrics[m], calculate_metric(
            m, search_results, qrels_results)]
        for m in evaluation_metrics
    ]
    )

    print(df)


## Articles containing "horas extraordinárias"

In [25]:
# you've got to do this manually
# for this example I searched for "horas extraordinárias" in the regular file
# and these IDs are "book/key/date"
qrels = [
    "Código Penal/58/1995-03-15T00:00:00Z",
    "Código Penal/58/2017-11-21T00:00:00Z",
    "Código Penal/58/2007-09-15T00:00:00Z",
]

# schemaless
schemaless_results = requests.get(
    SCHEMALESS_URL,
    params={
        "defType": "edismax",
        "qf": "title text path book",
        "pf": "title text path book",
        "q": "horas extraordinárias"
    }
).json()['response']['docs']

# schema with no weights
schema_simple_results = requests.get(
    SCHEMA_URL,
    params={
        "defType": "edismax",
        "qf": "title_raw title text_raw text path book",
        "pf": "title_raw title text_raw text path book",
        "q": "horas extraordinárias"
    }
).json()['response']['docs']

# schema and weight
schema_full_results = requests.get(
    SCHEMA_URL,
    params={
        "defType": "edismax",
        "qf": "title_raw^3 title^2 text_raw^1.5 text^1 path^0.5 book^0.25",
        "bq": "state:Consolidado^2", # FIXME changed to ^2 to give better results than the weightless (original was ^4)
        "pf": "title_raw^3 title^2 text_raw^1.5 text^1 path^0.5 book^0.25",
        "q": "horas extraordinárias"
    }
).json()['response']['docs']

# average precision takes the order into account
# precision at 10 is the percentage of relevant information in the top10
# recall is the percentage of relevant information in the total of results
metrics_table("Schemaless", schemaless_results, qrels)
metrics_table("Schema with no weights", schema_simple_results, qrels)
metrics_table("Schema with weights", schema_full_results, qrels)


Schemaless
                        0      1
0                  Metric  Value
1       Average Precision    0.0
2  Precision at 10 (P@10)    0.0
3                  Recall    0.0
Schema with no weights
                        0         1
0                  Metric     Value
1       Average Precision  0.665212
2  Precision at 10 (P@10)       0.3
3                  Recall       1.0
Schema with weights
                        0         1
0                  Metric     Value
1       Average Precision  0.665212
2  Precision at 10 (P@10)       0.3
3                  Recall       1.0


## Articles containing "horas extraordinárias" in book "Código do Trabalho"

It doesn't make sense, since the results are supposed to not be that correct

## "socialism" from the initial constituicao text (1976)

In [31]:
# you've got to do this manually
# for this example I searched for "horas extraordinárias" in the regular file
# and these IDs are "book/key/date"
qrels = [
    "Constituição da República Portuguesa/2/1976-04-10T00:00:00Z",
    "Constituição da República Portuguesa/89/1976-04-10T00:00:00Z",
    "Constituição da República Portuguesa/185/1976-04-10T00:00:00Z",
    "Constituição da República Portuguesa/273/1976-04-10T00:00:00Z",
]

# schemaless
schemaless_results = requests.get(
    SCHEMALESS_URL,
    params={
        "defType": "edismax",
        "qf": "title text path",
        "q": "socialism +date:[1976-01-01T00:00:00Z TO 1976-12-31T23:59:59Z] +book:constituicao",
    }
).json()['response']['docs']

# schema with no weights
schema_simple_results = requests.get(
    SCHEMA_URL,
    params={
        "defType": "edismax",
        "qf": "title_raw title text_raw text path",
        "q": "socialism +date:[1976-01-01T00:00:00Z TO 1976-12-31T23:59:59Z] +book:constituicao",
    }
).json()['response']['docs']

# schema and weight
schema_full_results = requests.get(
    SCHEMA_URL,
    params={
        "defType": "edismax",
        "qf": "title_raw^3 title^2 text_raw^1.5 text^1 path^0.5",
        "q": "socialism +date:[1976-01-01T00:00:00Z TO 1976-12-31T23:59:59Z] +book:constituicao",
    }
).json()['response']['docs']

# average precision takes the order into account
# precision at 10 is the percentage of relevant information in the top10
# recall is the percentage of relevant information in the total of results
metrics_table("Schemaless", schemaless_results, qrels)
metrics_table("Schema with no weights", schema_simple_results, qrels)
metrics_table("Schema with weights", schema_full_results, qrels)


Schemaless
                        0      1
0                  Metric  Value
1       Average Precision      0
2  Precision at 10 (P@10)    0.0
3                  Recall    0.0
Schema with no weights
                        0         1
0                  Metric     Value
1       Average Precision  0.775838
2  Precision at 10 (P@10)       0.4
3                  Recall       1.0
Schema with weights
                        0         1
0                  Metric     Value
1       Average Precision  0.775838
2  Precision at 10 (P@10)       0.4
3                  Recall       1.0
